In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import sqlite3

# CAPSTONE ETL PIPELINE
# Extract  → Transform → Load using PySpark

def etl_process(csv_file, db_file, table_name):

    """
    Capstone ETL Function
    ---------------------
    Extracts employee data from a CSV file, transforms it using PySpark,
    and loads the final dataset into a SQLite database.
    """


    # 1. SPARK SESSION INITIALIZATION

    # SparkSession is the main entry point for PySpark
    spark = SparkSession.builder \
        .appName("Week8_Capstone_ETL_Project") \
        .getOrCreate()

    try:

        # 2. EXTRACT

        # Load CSV data into a Spark DataFrame
        # inferSchema automatically detects column data types
        df = spark.read.csv(csv_file, header=True, inferSchema=True)

        print("=== Extracted Raw Data ===")
        df.show()


        # 3. TRANSFORM

        # Step 1: Remove records with missing values
        # This improves data quality and reliability
        cleaned_df = df.dropna()

        print("=== After Removing Null Values ===")
        cleaned_df.show()

        # Step 2: Rename columns for standard naming conventions
        transformed_df = cleaned_df \
            .withColumnRenamed("Name", "Employee Name") \
            .withColumnRenamed("Salary($)", "Salary")

        print("=== After Renaming Columns ===")
        transformed_df.show()

        # Step 3: Drop unnecessary or sensitive columns
        transformed_df = transformed_df.drop(
            "Email ID", "Joining Date", "Phone No"
        )

        print("=== After Dropping Unnecessary Columns ===")
        transformed_df.show()

        # Step 4: Add derived column using business logic
        # Calculate 10% salary hike for each employee
        transformed_df = transformed_df.withColumn(
            "Salary Hike",
            col("Salary") * 0.10
        )

        print("=== After Adding Salary Hike Column ===")
        transformed_df.show()


        # 4. LOAD

        # Convert Spark DataFrame to Pandas DataFrame
        # SQLite does not support direct Spark writes efficiently
        final_df = transformed_df.toPandas()

        # Connect to SQLite database
        conn = sqlite3.connect(db_file)

        # Load data into SQLite table
        final_df.to_sql(
            table_name,
            conn,
            if_exists="replace",
            index=False
        )

        conn.close()

        print(
            f"Data successfully loaded into database '{db_file}' "
            f"inside table '{table_name}'"
        )

        return final_df

    except Exception as e:
        print(f"Capstone ETL Pipeline Failed: {e}")
        raise

    finally:

        # 5. SPARK SESSION TERMINATION

        # Always stop Spark session after job completion
        spark.stop()





In [5]:

# CAPSTONE PIPELINE EXECUTION

if __name__ == "__main__":
    etl_process(
        csv_file="EmployeeData copy.csv",
        db_file="EmployeeData.db",
        table_name="EmployeeData"
    )

=== Extracted Raw Data ===
+---+------+---------------+----------+---------+------------+--------------------+----------+
|_c0|Emp ID|       FullName|      Dept|Salary($)|Joining Date|            Email ID|  Phone No|
+---+------+---------------+----------+---------+------------+--------------------+----------+
|  0|   101|   Alex Johnson|     Admin|  47508.0|  2017-01-27|alex.johnson@work...|9475287752|
|  1|   102|    Liam Wilson|        IT|     NULL|  2024-08-12|                NULL|9744732697|
|  2|   103|  Olivia Martin| Marketing|     NULL|  2021-02-08|olivia.martin@exa...|      NULL|
|  3|   104|  Olivia Martin|        IT|  75115.0|  2016-06-10|                NULL|      NULL|
|  4|   105|    Mason Young|     Admin|  83571.0|  2019-07-03|                NULL|9469220313|
|  5|   106| Benjamin Adams|   Finance|     NULL|  2023-09-28|benjamin.adams@wo...|      NULL|
|  6|   107|   Sophia Davis|        IT|  67812.0|  2022-05-15|                NULL|9242110844|
|  7|   108|   Rajesh K